# Final Project

By : 

Batch : HCK-001

## Import Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pylab as pl

from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
from matplotlib.ticker import FixedLocator, FixedFormatter
from feature_engine.outliers import Winsorizer
from feature_engine.outliers import Winsorizer

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture

from statsmodels.stats.outliers_influence import variance_inflation_factor 

# from clusteval import clusteval

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Data Loading

In [2]:
df_events = pd.read_csv("dsv1069_events.csv")
df_items = pd.read_csv("dsv1069_items.csv")
df_orders = pd.read_csv("dsv1069_orders.csv")
df_users = pd.read_csv("dsv1069_users.csv")

C:\Users\mahes\AppData\Local\Temp\ipykernel_15340\2536878048.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_events = pd.read_csv("dsv1069_events.csv")


In [3]:
df_events.head()

,event_id,event_time,user_id,event_name,platform,parameter_name,parameter_value
0,b9de71c5c3cc4cd7a97e50b832106e5a,2017-06-26 11:23:39,178481.0,view_item,android,item_id,3526
1,23267713c9ea44419331731f50b6a8db,2017-06-27 10:46:39,178481.0,view_item,android,item_id,1514
2,1b7822fa7b854e01970218ae8f721fe0,2017-06-27 11:15:39,178481.0,view_item,android,item_id,3712
3,2a7a188a626841ac94befcc419f06af4,2016-10-05 20:43:10,154133.0,view_item,android,item_id,3586
4,631d657264cc4616a4528f759509b25d,2016-10-04 03:29:10,154133.0,view_item,android,item_id,1061


events table will not be used for RFM analysis

In [4]:
df_items.head()

,adjective,category,created_at,id,modifier,name,price
0,fuzzy,contraption,2014-01-15 21:36:09,2512.0,carrying_case,fuzzy contraption carrying_case,150.0
1,NaN,instrument,2013-05-14 05:20:50,482.0,refill,instrument refill,35.2
2,industrial-strength,module,2014-02-04 19:28:32,2446.0,NaN,industrial-strength module,300.0
3,digital,tool,2013-02-25 12:23:18,1312.0,carrying_case,digital tool carrying_case,16.5
4,miniature,device,2013-08-05 17:20:45,3556.0,cleaner,miniature device cleaner,16.5


items table will not be used for RFM analysis

In [5]:
df_orders.head()

,invoice_id,line_item_id,user_id,item_id,item_name,item_category,price,created_at,paid_at
0,192320.0,83118.0,178481.0,3526.0,digital apparatus,apparatus,330.0,2017-06-28 21:14:25,2017-06-27 21:19:39
1,192320.0,207309.0,178481.0,1514.0,miniature apparatus cleaner,apparatus,99.0,2017-06-28 21:14:25,2017-06-27 21:19:39
2,192320.0,392027.0,178481.0,3712.0,miniature apparatus cleaner,apparatus,99.0,2017-06-28 21:14:25,2017-06-27 21:19:39
3,80902.0,243831.0,154133.0,3586.0,reflective instrument,instrument,57.2,2016-10-09 06:57:30,2016-10-07 10:08:10
4,80902.0,399806.0,154133.0,1061.0,extra-strength instrument charger,instrument,17.6,2016-10-09 06:57:30,2016-10-07 10:08:10


* invoice_id can be used to find total order, 
* user_id for unique id,
* item_id can be used to find total item bought, 
* price for monetary value 
* paid_at for latest transaction

In [6]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47402 entries, 0 to 47401
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   invoice_id     47402 non-null  float64
 1   line_item_id   47402 non-null  float64
 2   user_id        47402 non-null  float64
 3   item_id        47402 non-null  float64
 4   item_name      47402 non-null  object 
 5   item_category  47402 non-null  object 
 6   price          47402 non-null  float64
 7   created_at     47402 non-null  object 
 8   paid_at        47402 non-null  object 
dtypes: float64(5), object(4)
memory usage: 3.3+ MB


created_at and paid_at are still object, will be converted to datetime later

In [7]:
df_users.head()

,created_at,deleted_at,email_address,first_name,id,last_name,merged_at,parent_user_id
0,2014-12-20 07:07:45,NaN,ArataHopper@earthlink.info,Arata,51590.0,Hopper,NaN,NaN
1,2016-10-14 05:39:20,NaN,Riya_Gruber1974@mail.net,Riya,158891.0,Gruber,NaN,NaN
2,2017-01-21 10:20:09,NaN,Peter_Sousa@yahoo.info,Peter,179949.0,Sousa,NaN,NaN
3,2015-10-30 21:31:30,NaN,D_Kowalski1962@gmail.com,D,98000.0,Kowalski,NaN,NaN
4,2015-10-24 16:27:27,NaN,SNovak1966@mail.com,S,93994.0,Novak,NaN,NaN


* created_at can be used to find when was the id created

In [8]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117178 entries, 0 to 117177
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   created_at      117178 non-null  object 
 1   deleted_at      2888 non-null    object 
 2   email_address   117178 non-null  object 
 3   first_name      117178 non-null  object 
 4   id              117178 non-null  float64
 5   last_name       117178 non-null  object 
 6   merged_at       6408 non-null    object 
 7   parent_user_id  6408 non-null    float64
dtypes: float64(2), object(6)
memory usage: 7.2+ MB


created at will be converted to datetime

## RFM Analysis

In this RFM analysis we will be doing the following:
* Recency: calculate the length in terms of days, between latest date and last purchase of a customer
* Frequency: calculate the amount of invoice for each customer
* Monetary: calculate the total amount of money spent from a customer
* Length: calculate the total days between the creation of the ID and the latest order of the ID

### Merge Orders and Users

In [9]:
new_df = df_orders.merge(df_users,how='inner',left_on="user_id",right_on='id',suffixes=('_orders', '_users'))
new_df

,invoice_id,line_item_id,user_id,item_id,item_name,item_category,price,created_at_orders,paid_at,created_at_users,deleted_at,email_address,first_name,id,last_name,merged_at,parent_user_id
0,192320.0,83118.0,178481.0,3526.0,digital apparatus,apparatus,330.000,2017-06-28 21:14:25,2017-06-27 21:19:39,2017-01-02 04:17:55,NaN,ADavis1973@gmail.edu,A,178481.0,Davis,NaN,NaN
1,192320.0,207309.0,178481.0,1514.0,miniature apparatus cleaner,apparatus,99.000,2017-06-28 21:14:25,2017-06-27 21:19:39,2017-01-02 04:17:55,NaN,ADavis1973@gmail.edu,A,178481.0,Davis,NaN,NaN
2,192320.0,392027.0,178481.0,3712.0,miniature apparatus cleaner,apparatus,99.000,2017-06-28 21:14:25,2017-06-27 21:19:39,2017-01-02 04:17:55,NaN,ADavis1973@gmail.edu,A,178481.0,Davis,NaN,NaN
3,80902.0,243831.0,154133.0,3586.0,reflective instrument,instrument,57.200,2016-10-09 06:57:30,2016-10-07 10:08:10,2016-09-09 05:30:45,NaN,Anne_Davis1985@mail.edu,Anne,154133.0,Davis,NaN,NaN
4,80902.0,399806.0,154133.0,1061.0,extra-strength instrument charger,instrument,17.600,2016-10-09 06:57:30,2016-10-07 10:08:10,2016-09-09 05:30:45,NaN,Anne_Davis1985@mail.edu,Anne,154133.0,Davis,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47397,92108.0,236314.0,230808.0,1677.0,rechargable widget,widget,3.900,2017-12-04 19:01:55,2017-12-05 03:00:14,2017-09-13 08:59:57,NaN,LisaRodriquez@outlook.edu,Lisa,230808.0,Rodriquez,NaN,NaN
47398,348144.0,162528.0,151538.0,3729.0,prize-winning gadget wrapper,gadget,39.160,2016-11-18 21:00:07,2016-11-18 14:19:52,2016-09-01 22:55:04,NaN,Lisa_Santos@yahoo.net,Lisa,151538.0,Santos,NaN,NaN
47399,348144.0,321219.0,151538.0,1645.0,digital gadget carrying_case,gadget,58.740,2016-11-18 21:00:07,2016-11-18 14:19:52,2016-09-01 22:55:04,NaN,Lisa_Santos@yahoo.net,Lisa,151538.0,Santos,NaN,NaN
47400,348144.0,264112.0,151538.0,2084.0,matte gadget storage_unit,gadget,278.125,2016-11-18 21:00:07,2016-11-18 14:19:52,2016-09-01 22:55:04,NaN,Lisa_Santos@yahoo.net,Lisa,151538.0,Santos,NaN,NaN


### Drop Unused Columns

In [10]:
new_df.drop(['deleted_at','email_address','first_name','last_name','merged_at','parent_user_id','id','item_name','item_category','line_item_id','created_at_orders'],axis=1,inplace=True)
new_df

,invoice_id,user_id,item_id,price,paid_at,created_at_users
0,192320.0,178481.0,3526.0,330.000,2017-06-27 21:19:39,2017-01-02 04:17:55
1,192320.0,178481.0,1514.0,99.000,2017-06-27 21:19:39,2017-01-02 04:17:55
2,192320.0,178481.0,3712.0,99.000,2017-06-27 21:19:39,2017-01-02 04:17:55
3,80902.0,154133.0,3586.0,57.200,2016-10-07 10:08:10,2016-09-09 05:30:45
4,80902.0,154133.0,1061.0,17.600,2016-10-07 10:08:10,2016-09-09 05:30:45
...,...,...,...,...,...,...
47397,92108.0,230808.0,1677.0,3.900,2017-12-05 03:00:14,2017-09-13 08:59:57
47398,348144.0,151538.0,3729.0,39.160,2016-11-18 14:19:52,2016-09-01 22:55:04
47399,348144.0,151538.0,1645.0,58.740,2016-11-18 14:19:52,2016-09-01 22:55:04
47400,348144.0,151538.0,2084.0,278.125,2016-11-18 14:19:52,2016-09-01 22:55:04


### To Datetime

In [11]:
new_df['paid_at'] = pd.to_datetime(new_df['paid_at'])
new_df['created_at_users'] = pd.to_datetime(new_df['created_at_users'])

In [12]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47402 entries, 0 to 47401
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   invoice_id        47402 non-null  float64       
 1   user_id           47402 non-null  float64       
 2   item_id           47402 non-null  float64       
 3   price             47402 non-null  float64       
 4   paid_at           47402 non-null  datetime64[ns]
 5   created_at_users  47402 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(4)
memory usage: 2.5 MB


### Group By Invoice ID

In [13]:
df_agg = new_df.groupby('invoice_id',
            as_index=False).agg(user_id=('user_id',np.max),
                                total_price=('price',np.sum),
                                paid_at=('paid_at',np.max),
                                created_at=('created_at_users',np.min)
                                )
df_agg

,invoice_id,user_id,total_price,paid_at,created_at
0,6.0,89601.0,143.00,2016-03-13 18:38:19,2015-09-14 14:21:56
1,20.0,50878.0,177.22,2015-03-25 03:11:09,2014-12-29 08:52:28
2,30.0,69737.0,0.00,2015-11-08 15:07:15,2015-05-06 04:36:50
3,64.0,139762.0,15.75,2016-12-30 03:33:16,2016-06-20 14:16:04
4,98.0,11072.0,178.64,2014-05-02 03:58:15,2013-11-14 14:23:26
...,...,...,...,...,...
18966,399938.0,252227.0,300.00,2018-03-13 11:46:58,2017-11-26 22:36:14
18967,399943.0,201699.0,1080.00,2017-05-19 01:51:42,2017-04-23 13:31:26
18968,399960.0,86432.0,105.60,2016-01-30 16:14:37,2015-08-25 04:06:27
18969,399979.0,198052.0,32.04,2017-08-24 11:23:24,2017-04-23 08:00:06


In [14]:
total_item = new_df.groupby(['invoice_id'],as_index=False)['item_id'].count()
total_item.rename(columns={'item_id':'total_items'},inplace=True)
total_item.sort_values(by='total_items',ascending=True)

,invoice_id,total_items
0,6.0,1
8807,185759.0,1
8806,185747.0,1
8804,185713.0,1
8800,185642.0,1
...,...,...
3259,69305.0,4
3258,69297.0,4
13603,286959.0,4
13600,286912.0,4


In [15]:
by_invoice = df_agg.merge(total_item,how='inner',left_on='invoice_id',right_on='invoice_id')
by_invoice

,invoice_id,user_id,total_price,paid_at,created_at,total_items
0,6.0,89601.0,143.00,2016-03-13 18:38:19,2015-09-14 14:21:56,1
1,20.0,50878.0,177.22,2015-03-25 03:11:09,2014-12-29 08:52:28,3
2,30.0,69737.0,0.00,2015-11-08 15:07:15,2015-05-06 04:36:50,1
3,64.0,139762.0,15.75,2016-12-30 03:33:16,2016-06-20 14:16:04,2
4,98.0,11072.0,178.64,2014-05-02 03:58:15,2013-11-14 14:23:26,4
...,...,...,...,...,...,...
18966,399938.0,252227.0,300.00,2018-03-13 11:46:58,2017-11-26 22:36:14,1
18967,399943.0,201699.0,1080.00,2017-05-19 01:51:42,2017-04-23 13:31:26,3
18968,399960.0,86432.0,105.60,2016-01-30 16:14:37,2015-08-25 04:06:27,3
18969,399979.0,198052.0,32.04,2017-08-24 11:23:24,2017-04-23 08:00:06,1


### Group By User ID (Frequency and Monetary)

In [16]:
by_id = by_invoice.groupby('user_id',as_index=False).agg(
                                Monetary=('total_price',np.sum),
                                last_order=('paid_at',np.max),
                                created_at=('created_at',np.min),
                                Amount=('total_items',np.sum)
                                )
by_id

,user_id,Monetary,last_order,created_at,Amount
0,17.0,243.00,2013-05-25 08:27:17,2013-03-03 02:42:42,3
1,25.0,1.80,2013-05-17 03:11:48,2013-03-06 16:01:06,2
2,63.0,585.00,2013-07-07 11:26:45,2013-03-12 16:34:26,3
3,65.0,44.50,2013-05-09 10:22:58,2013-03-08 01:55:17,1
4,69.0,9.42,2013-08-09 23:25:51,2013-02-25 01:26:18,4
...,...,...,...,...,...
17458,295368.0,45.00,2018-06-01 09:27:57,2018-05-16 20:53:54,1
17459,296332.0,4.65,2018-05-29 22:55:55,2018-05-19 14:15:06,2
17460,296584.0,8.82,2018-06-02 15:15:39,2018-05-17 21:27:18,4
17461,296887.0,144.00,2018-05-29 15:02:38,2018-05-20 06:09:25,4


In [17]:
frequency = by_invoice.groupby('user_id',as_index=False)['invoice_id'].count()
frequency.rename(columns={'invoice_id':'Frequency'},inplace=True)
frequency.sort_values(by='Frequency', ascending=True)

,user_id,Frequency
0,17.0,1
11403,176297.0,1
11404,176323.0,1
11405,176325.0,1
11406,176373.0,1
...,...,...
12290,191026.0,4
15058,234728.0,4
11763,182241.0,4
5717,88724.0,4


In [18]:
df_frequency = by_id.merge(frequency,how='inner',left_on='user_id',right_on='user_id')
df_frequency

,user_id,Monetary,last_order,created_at,Amount,Frequency
0,17.0,243.00,2013-05-25 08:27:17,2013-03-03 02:42:42,3,1
1,25.0,1.80,2013-05-17 03:11:48,2013-03-06 16:01:06,2,1
2,63.0,585.00,2013-07-07 11:26:45,2013-03-12 16:34:26,3,1
3,65.0,44.50,2013-05-09 10:22:58,2013-03-08 01:55:17,1,1
4,69.0,9.42,2013-08-09 23:25:51,2013-02-25 01:26:18,4,1
...,...,...,...,...,...,...
17458,295368.0,45.00,2018-06-01 09:27:57,2018-05-16 20:53:54,1,1
17459,296332.0,4.65,2018-05-29 22:55:55,2018-05-19 14:15:06,2,1
17460,296584.0,8.82,2018-06-02 15:15:39,2018-05-17 21:27:18,4,1
17461,296887.0,144.00,2018-05-29 15:02:38,2018-05-20 06:09:25,4,1


### Recency

In [19]:
# remove timestamp
df_frequency['last_order'] = df_frequency['last_order'].dt.date
df_frequency['created_at'] = df_frequency['created_at'].dt.date

In [20]:
df_frequency

,user_id,Monetary,last_order,created_at,Amount,Frequency
0,17.0,243.00,2013-05-25,2013-03-03,3,1
1,25.0,1.80,2013-05-17,2013-03-06,2,1
2,63.0,585.00,2013-07-07,2013-03-12,3,1
3,65.0,44.50,2013-05-09,2013-03-08,1,1
4,69.0,9.42,2013-08-09,2013-02-25,4,1
...,...,...,...,...,...,...
17458,295368.0,45.00,2018-06-01,2018-05-16,1,1
17459,296332.0,4.65,2018-05-29,2018-05-19,2,1
17460,296584.0,8.82,2018-06-02,2018-05-17,4,1
17461,296887.0,144.00,2018-05-29,2018-05-20,4,1


In [21]:
# calculating latest purchase
MyList = []
for i in range(len(df_frequency)):
    recent = df_frequency['last_order'].max() - df_frequency['last_order'][i]
    MyList.append(recent)

In [22]:
recency = pd.DataFrame(MyList,columns=["Recency"])
recency

,Recency
0,1836 days
1,1844 days
2,1793 days
3,1852 days
4,1760 days
...,...
17458,3 days
17459,6 days
17460,2 days
17461,6 days


In [23]:
df_recency = pd.concat([df_frequency,recency],axis=1)
df_recency

,user_id,Monetary,last_order,created_at,Amount,Frequency,Recency
0,17.0,243.00,2013-05-25,2013-03-03,3,1,1836 days
1,25.0,1.80,2013-05-17,2013-03-06,2,1,1844 days
2,63.0,585.00,2013-07-07,2013-03-12,3,1,1793 days
3,65.0,44.50,2013-05-09,2013-03-08,1,1,1852 days
4,69.0,9.42,2013-08-09,2013-02-25,4,1,1760 days
...,...,...,...,...,...,...,...
17458,295368.0,45.00,2018-06-01,2018-05-16,1,1,3 days
17459,296332.0,4.65,2018-05-29,2018-05-19,2,1,6 days
17460,296584.0,8.82,2018-06-02,2018-05-17,4,1,2 days
17461,296887.0,144.00,2018-05-29,2018-05-20,4,1,6 days


### Length

In [24]:
# calculating total days as customer
MyList2 = []
for i in range(len(df_recency)):
    days = df_recency['last_order'][i] - df_recency['created_at'][i]
    MyList2.append(days)

In [25]:
length = pd.DataFrame(MyList2,columns=["Length"])
length

,Length
0,83 days
1,72 days
2,117 days
3,62 days
4,165 days
...,...
17458,16 days
17459,10 days
17460,16 days
17461,9 days


In [26]:
df_length = pd.concat([df_recency,length],axis=1)
df_length

,user_id,Monetary,last_order,created_at,Amount,Frequency,Recency,Length
0,17.0,243.00,2013-05-25,2013-03-03,3,1,1836 days,83 days
1,25.0,1.80,2013-05-17,2013-03-06,2,1,1844 days,72 days
2,63.0,585.00,2013-07-07,2013-03-12,3,1,1793 days,117 days
3,65.0,44.50,2013-05-09,2013-03-08,1,1,1852 days,62 days
4,69.0,9.42,2013-08-09,2013-02-25,4,1,1760 days,165 days
...,...,...,...,...,...,...,...,...
17458,295368.0,45.00,2018-06-01,2018-05-16,1,1,3 days,16 days
17459,296332.0,4.65,2018-05-29,2018-05-19,2,1,6 days,10 days
17460,296584.0,8.82,2018-06-02,2018-05-17,4,1,2 days,16 days
17461,296887.0,144.00,2018-05-29,2018-05-20,4,1,6 days,9 days


In [27]:
df_RFM = df_length[['user_id','Recency','Frequency','Monetary','Length']]
df_RFM

,user_id,Recency,Frequency,Monetary,Length
0,17.0,1836 days,1,243.00,83 days
1,25.0,1844 days,1,1.80,72 days
2,63.0,1793 days,1,585.00,117 days
3,65.0,1852 days,1,44.50,62 days
4,69.0,1760 days,1,9.42,165 days
...,...,...,...,...,...
17458,295368.0,3 days,1,45.00,16 days
17459,296332.0,6 days,1,4.65,10 days
17460,296584.0,2 days,1,8.82,16 days
17461,296887.0,6 days,1,144.00,9 days


### RFM EDA

In [28]:
df_RFM['Frequency'].value_counts()

1    16042
2     1339
3       77
4        5
Name: Frequency, dtype: int64

In [29]:
df_RFM[['Recency','Frequency','Monetary','Length']].describe()

,Recency,Frequency,Monetary,Length
count,17463,17463.000000,17463.000000,17463
mean,688 days 19:58:23.521731656,1.086354,276.381676,100 days 14:15:11.595945714
std,462 days 06:41:08.242346496,0.299063,383.592673,58 days 05:54:31.621686840
min,0 days 00:00:00,1.000000,0.000000,-3 days +00:00:00
25%,293 days 00:00:00,1.000000,43.200000,50 days 00:00:00
50%,629 days 00:00:00,1.000000,126.000000,100 days 00:00:00
75%,1035 days 00:00:00,1.000000,338.280000,151 days 00:00:00
max,1908 days 00:00:00,4.000000,4027.500000,205 days 00:00:00


We will be doing manual scoring/ranking of the RFML by using quantiles. Lower scores is better, while higher score is of lower performance/rank

In [30]:
RFM_segment = df_RFM.copy()

In [31]:
recency_score = []
for i in range(len(df_RFM)):
    if df_RFM['Recency'][i] <= df_RFM['Recency'].quantile(0.25):
        recency_score.append(1)
    elif df_RFM['Recency'][i] <= df_RFM['Recency'].quantile(0.5):
        recency_score.append(2)
    elif df_RFM['Recency'][i] <= df_RFM['Recency'].quantile(0.75):
        recency_score.append(3)
    else:
        recency_score.append(4)

recency_score = pd.DataFrame(recency_score,columns=["R_score"])
recency_score

,R_score
0,4
1,4
2,4
3,4
4,4
...,...
17458,1
17459,1
17460,1
17461,1


In [32]:
frequency_score = []
for i in range(len(df_RFM)):
    if df_RFM['Frequency'][i] <= df_RFM['Frequency'].quantile(0.25):
        frequency_score.append(4)
    elif df_RFM['Frequency'][i] <= df_RFM['Frequency'].quantile(0.5):
        frequency_score.append(3)
    elif df_RFM['Frequency'][i] <= df_RFM['Frequency'].quantile(0.75):
        frequency_score.append(2)
    else:
        frequency_score.append(1)

frequency_score = pd.DataFrame(frequency_score,columns=["F_score"])
frequency_score

,F_score
0,4
1,4
2,4
3,4
4,4
...,...
17458,4
17459,4
17460,4
17461,4


In [33]:
monetary_score = []
for i in range(len(df_RFM)):
    if df_RFM['Monetary'][i] <= df_RFM['Monetary'].quantile(0.25):
        monetary_score.append(4)
    elif df_RFM['Monetary'][i] <= df_RFM['Monetary'].quantile(0.5):
        monetary_score.append(3)
    elif df_RFM['Monetary'][i] <= df_RFM['Monetary'].quantile(0.75):
        monetary_score.append(2)
    else:
        monetary_score.append(1)

monetary_score = pd.DataFrame(monetary_score,columns=["M_score"])
monetary_score

,M_score
0,2
1,4
2,1
3,3
4,4
...,...
17458,3
17459,4
17460,4
17461,2


In [34]:
length_score = []
for i in range(len(df_RFM)):
    if df_RFM['Length'][i] <= df_RFM['Length'].quantile(0.25):
        length_score.append(4)
    elif df_RFM['Length'][i] <= df_RFM['Length'].quantile(0.5):
        length_score.append(3)
    elif df_RFM['Length'][i] <= df_RFM['Length'].quantile(0.75):
        length_score.append(2)
    else:
        length_score.append(1)

length_score = pd.DataFrame(length_score,columns=["L_score"])
length_score

,L_score
0,3
1,3
2,2
3,3
4,1
...,...
17458,4
17459,4
17460,4
17461,4


In [35]:
segments = []
for i in range(len(df_RFM)):
    n = str(recency_score['R_score'][i]) + str(frequency_score['F_score'][i]) + str(monetary_score['M_score'][i]) + str(length_score['L_score'][i])
    segments.append(n)

segments = pd.DataFrame(segments,columns=["RFM_segment"])
segments

,RFM_segment
0,4423
1,4443
2,4412
3,4433
4,4441
...,...
17458,1434
17459,1444
17460,1444
17461,1424


In [36]:
scores = []
for i in range(len(df_RFM)):
    n = recency_score['R_score'][i] + frequency_score['F_score'][i] + monetary_score['M_score'][i] + length_score['L_score'][i]
    scores.append(n)

scores = pd.DataFrame(scores,columns=["RFM_score"])
scores

,RFM_score
0,13
1,15
2,11
3,14
4,13
...,...
17458,12
17459,13
17460,13
17461,11


In [37]:
RFM_segment = pd.concat([df_RFM,recency_score,frequency_score,monetary_score,length_score,segments,scores],axis=1)
RFM_segment

,user_id,Recency,Frequency,Monetary,Length,R_score,F_score,M_score,L_score,RFM_segment,RFM_score
0,17.0,1836 days,1,243.00,83 days,4,4,2,3,4423,13
1,25.0,1844 days,1,1.80,72 days,4,4,4,3,4443,15
2,63.0,1793 days,1,585.00,117 days,4,4,1,2,4412,11
3,65.0,1852 days,1,44.50,62 days,4,4,3,3,4433,14
4,69.0,1760 days,1,9.42,165 days,4,4,4,1,4441,13
...,...,...,...,...,...,...,...,...,...,...,...
17458,295368.0,3 days,1,45.00,16 days,1,4,3,4,1434,12
17459,296332.0,6 days,1,4.65,10 days,1,4,4,4,1444,13
17460,296584.0,2 days,1,8.82,16 days,1,4,4,4,1444,13
17461,296887.0,6 days,1,144.00,9 days,1,4,2,4,1424,11
